In [2]:
import numpy as np
from functools import wraps
from time import time

In [4]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap


In [6]:
S0 = 100
K = 100
T = 1
r= 0.06
N = 3
u = 1.1
d = 1/u
opttype = 'C'

In [8]:
@timing
def binomial_tree_slow(K, T, S0, r, N, u, d, opttype='C'):
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    S = np.zeros(N+1)
    S[0] = S0*(d**N)
    for j in range(1, N+1):
        S[j] = S[j-1]*u/d

    C = np.zeros(N+1)
    for j in range(0, N+1):
        C[j] = max(0, S[j]-K)

    for i in np.arange(N, 0, -1):
        for j in range(0, i):
            C[j] = disc * (q*C[j+1] + (1-q)*C[j])

    return C[0]

price = binomial_tree_slow(K, T, S0, r, N, u, d, opttype='C')
print(price)

func:'binomial_tree_slow' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0001 sec
10.145735799928817


In [11]:
@timing
def binomial_tree_fast(K, T, S0, r, N, u, d, opttype='C'):
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    C = S0*(d**(np.arange(N, -1, -1)))*(u**(np.arange(0, N+1, 1)))
    C = np.maximum(C-K, np.zeros(N+1))

    for i in np.arange(N, 0, -1):
        C = disc*(q*C[1:i+1] + (1-q)*C[0:i])

    return C[0]

price = binomial_tree_fast(K, T, S0, r, N, u, d, opttype='C')
print(price)

func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0001 sec
10.145735799928826


In [12]:
for N in [3,50, 100, 1000, 5000]:
    binomial_tree_slow(K,T,S0,r,N,u,d,opttype='C')
    binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')

func:'binomial_tree_slow' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0001 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0003 sec
func:'binomial_tree_slow' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0007 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0003 sec
func:'binomial_tree_slow' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0027 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0006 sec
func:'binomial_tree_slow' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.2547 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0059 sec
func:'binomial_tree_slow' args:[(100, 1, 100, 0.06, 5000, 1.